# Feasibility cuts

Ref: Birge and Louveaux, "Introduction to Stochastic Programming"

In [1]:
using JuMP
using Gurobi

INFO: Recompiling stale cache file C:\Users\bastin\.julia\lib\v0.6\ReverseDiffSparse.ji for module ReverseDiffSparse.
INFO: Recompiling stale cache file C:\Users\bastin\.julia\lib\v0.6\JuMP.ji for module JuMP.
INFO: Recompiling stale cache file C:\Users\bastin\.julia\lib\v0.6\Gurobi.ji for module Gurobi.


We consider the problem
\begin{align*}
    \min_x\ & x_1 + x_2 + E[Q(x)]\\
    \mbox{s.t. } & x \geq 0
\end{align*}
where
\begin{align*}
Q(x) = \min_y & -15y_1 -12y_2 \\
\mbox{s.t. } & 3y_1+ 2y_2 \leq x_1 \\
& 2y_1 + 5y_2 \leq x_2 \\
& 0.8\xi_1 \leq y_1 \leq \xi_1 \\
& 0.8\xi_2 \leq y_2 \leq \xi_2 \\
& y \geq 0
\end{align*}


In [3]:
nScens = 4
pScenario = 1/nScens
p = [pScenario for i = 1:nScens]
first = Model(solver = GurobiSolver())
@variable(first,x[1:2] >= 0)

2-element Array{JuMP.Variable,1}:
 x[1]
 x[2]

In [18]:
function secondStage(x::Vector, ξ::Vector)
    m = Model(solver = GurobiSolver())

    @variable(m,y[1:2])

    @constraint(m, ressource1, 3y[1]+2y[2] <= x[1])
    @constraint(m, ressource2, 2y[1]+5y[2] <= x[2])
    @constraint(m, lb1, y[1] >= 0.8ξ[1])
    @constraint(m, ub1, y[1] <= ξ[1])
    @constraint(m, lb2, y[2] >= 0.8ξ[2])
    @constraint(m, ub2, y[2] <= ξ[2])

    @objective(m,Min, -15y[1]-12y[2])
    
    status = solve(m)
    
    return m, status
end

secondStage (generic function with 1 method)

In [24]:
function secondStage_artificial(x::Vector, ξ::Vector)
    m = Model(solver = GurobiSolver())

    @variable(m,y[1:2])
    @variable(m, splus[1:6] >= 0)
    @variable(m, sminus[1:6] >= 0)

    @constraint(m, ressource1, 3y[1]+2y[2]+splus[1]-sminus[1] <= x[1])
    @constraint(m, ressource2, 2y[1]+5y[2]+splus[2]-sminus[2] <= x[2])
    @constraint(m, lb1, y[1]+splus[3]-sminus[3] >= 0.8ξ[1])
    @constraint(m, ub1, y[1]+splus[4]-sminus[4] <= ξ[1])
    @constraint(m, lb2, y[2]+splus[5]-sminus[5] >= 0.8ξ[2])
    @constraint(m, ub2, y[2]+splus[6]-sminus[6] <= ξ[2])

    @objective(m,Min, sum(splus[i] for i=1:6)+sum(sminus[i] for i=1:6))
    
    status = solve(m)

    dual = zeros(6)
    dual[1] = getdual(ressource1)
    dual[2] = getdual(ressource2)
    dual[3] = getdual(lb1)
    dual[4] = getdual(ub1)
    dual[5] = getdual(lb2)
    dual[6] = getdual(ub2)

    return m, status, dual
end

secondStage_artificial (generic function with 1 method)

Note: some of the introduced artifical variables are useless.

In [25]:
ξ = [6,8]
m, status = secondStage([0,0],ξ)

Academic license - for non-commercial use only
Optimize a model with 6 rows, 2 columns and 8 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [1e+01, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 8e+00]
Presolve removed 4 rows and 0 columns
Presolve time: 0.00s
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0      handle free variables                          0s

Solved in 5 iterations and 0.00 seconds
Infeasible model


(Minimization problem with:
 * 6 linear constraints
 * 2 variables
Solver is Gurobi, :Infeasible)

In [26]:
status

:Infeasible

In [27]:
if (status == :Infeasible)
    # We have to build the artificial problem
    martificial, status, dual = secondStage_artificial([0,0],[6,8])
end

(Minimization problem with:
 * 6 linear constraints
 * 14 variables
Solver is Gurobi, :Optimal, [-0.272727, -0.0909091, 1.0, 0.0, 1.0, 0.0])

Academic license - for non-commercial use only
Optimize a model with 6 rows, 14 columns and 20 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 8e+00]
Presolve removed 4 rows and 12 columns
Presolve time: 0.00s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.200000e+01   0.000000e+00      0s
       2    1.1200000e+01   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.00 seconds
Optimal objective  1.120000000e+01


In [28]:
print(martificial)

Min splus[1] + splus[2] + splus[3] + splus[4] + splus[5] + splus[6] + sminus[1] + sminus[2] + sminus[3] + sminus[4] + sminus[5] + sminus[6]
Subject to
 3 y[1] + 2 y[2] + splus[1] - sminus[1] <= 0
 2 y[1] + 5 y[2] + splus[2] - sminus[2] <= 0
 y[1] + splus[3] - sminus[3] >= 4.800000000000001
 y[1] + splus[4] - sminus[4] <= 6
 y[2] + splus[5] - sminus[5] >= 6.4
 y[2] + splus[6] - sminus[6] <= 8
 y[i] for all i in {1,2}
 splus[i] >= 0 for all i in {1,2,..,5,6}
 sminus[i] >= 0 for all i in {1,2,..,5,6}


In [29]:
getobjectivevalue(martificial)

11.200000000000001

In [30]:
dual

6-element Array{Float64,1}:
 -0.272727 
 -0.0909091
  1.0      
  0.0      
  1.0      
  0.0      

In [31]:
h = [0 ; 0 ; 0.8*ξ[1]; ξ[1]; 0.8*ξ[2]; ξ[2]]
T = [-1.0 0 ; 0 -1.0; 0 0 ; 0 0 ; 0 0; 0 0]
D = dual'*T
d = dot(dual,h)

11.200000000000001

In [32]:
@constraint(first, cut, sum(D[i]*x[i] for i = 1:2) >= d)

0.2727272727272727 x[1] + 0.09090909090909093 x[2] >= 11.200000000000001

In [33]:
first

Feasibility problem with:
 * 1 linear constraint
 * 2 variables
Solver is Gurobi

In [36]:
function secondStage_artificial_improved(x::Vector, ξ::Vector)
    m = Model(solver = GurobiSolver())

    @variable(m,y[1:2])
    @variable(m, splus[1:6] >= 0)
    @variable(m, sminus[1:6] >= 0)

    @constraintref artCons[1:6]
    
    artCons[1] = @constraint(m, 3y[1]+2y[2]+splus[1]-sminus[1] <= x[1])
    artCons[2] = @constraint(m, 2y[1]+5y[2]+splus[2]-sminus[2] <= x[2])
    artCons[3] = @constraint(m, y[1]+splus[3]-sminus[3] >= 0.8ξ[1])
    artCons[4] = @constraint(m, y[1]+splus[4]-sminus[4] <= ξ[1])
    artCons[5] = @constraint(m, y[2]+splus[5]-sminus[5] >= 0.8ξ[2])
    artCons[6] = @constraint(m, y[2]+splus[6]-sminus[6] <= ξ[2])

    @objective(m,Min, sum(splus[i] for i=1:6)+sum(sminus[i] for i=1:6))
    
    status = solve(m)

    dual = getdual(artCons)

    return m, status, dual
end

secondStage_artificial_improved (generic function with 1 method)

In [37]:
martificial, status, dual = secondStage_artificial_improved([0,0],[6,8])

(Minimization problem with:
 * 6 linear constraints
 * 14 variables
Solver is Gurobi, :Optimal, [-0.272727, -0.0909091, 1.0, 0.0, 1.0, 0.0])

Academic license - for non-commercial use only
Optimize a model with 6 rows, 14 columns and 20 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 8e+00]
Presolve removed 4 rows and 12 columns
Presolve time: 0.00s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.200000e+01   0.000000e+00      0s
       2    1.1200000e+01   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.00 seconds
Optimal objective  1.120000000e+01


In [38]:
dual

6-element Array{Float64,1}:
 -0.272727 
 -0.0909091
  1.0      
  0.0      
  1.0      
  0.0      